<a href="https://colab.research.google.com/github/aravind-chilakamarri/CS5001-Fundamentals-of-Data-Engineering/blob/main/Project_7_Python_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
#imports and initializations
import requests
import pandas as pd
import sqlite3
connection = sqlite3.connect('brewery_data.db')
fetch_data = []

In [49]:
# initialize api endpoint
api = "https://api.openbrewerydb.org/v1/breweries/random?size=50"

In [50]:
#check response
response = requests.get(api)

In [51]:
#check the count given by response
len(response.json())

50

In [52]:
# hit the api as many time till you get 10000 records
for i in range(10000//len(response.json())):
  response = requests.get(api)
  fetch_data.extend(response.json())

In [53]:
#check number of fetched records which should match 10000
len(fetch_data)

10000

In [54]:
# check what kind of records in fetch_data list. Seems like a dict element
fetch_data[0]

{'id': '8b44ff77-fb30-4b24-afa6-844d9af5c8af',
 'name': 'Carver Brewing Co',
 'brewery_type': 'brewpub',
 'address_1': '1022 Main Ave',
 'address_2': None,
 'address_3': None,
 'city': 'Durango',
 'state_province': 'Colorado',
 'postal_code': '81301-5124',
 'country': 'United States',
 'longitude': '-107.8800415',
 'latitude': '37.2748415',
 'phone': '9702592545',
 'website_url': 'http://carverbrewing.com',
 'state': 'Colorado',
 'street': '1022 Main Ave'}

In [55]:
# prepare the dataframe from fetch_data list
df = pd.DataFrame(fetch_data)

In [56]:
df.shape

(10000, 16)

In [57]:
# first remove dups
# next replace NA with Phone number unknown in phone column
# next replace NA with website url unknown in website_url column

df.drop_duplicates(inplace=True)
df['phone'].fillna("Phone number unknown", inplace=True)
df['website_url'].fillna("Website URL unknown", inplace=True)

In [58]:
# check the shape after data transformations
df.shape

(5741, 16)

In [59]:
# just a  quick check
df.iterrows()

<generator object DataFrame.iterrows at 0x7c15563a66c0>

In [61]:
# create a cusror from created connection and start loading the data
cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS US_data (id INTEGER PRIMARY KEY, name TEXT, city TEXT, state TEXT, country TEXT, phone TEXT)')
cursor.execute('CREATE TABLE IF NOT EXISTS Non_US_data (id INTEGER PRIMARY KEY, name TEXT, city TEXT, state TEXT, country TEXT, phone TEXT)')

# Load the transformed data into the SQLite tables based on the country
us_data = df[df['country'] == 'United States']
non_us_data = df[df['country'] != 'United States']

# Insert data into the 'US_data' table
us_data.to_sql('US_data', connection, if_exists='replace', index=False)

# Insert data into the 'Non_US_data' table
non_us_data.to_sql('Non_US_data', connection, if_exists='replace', index=False)

# Commit the changes and close the connection
connection.commit()


us_data = pd.read_sql_query("SELECT * FROM US_data", connection)
non_us_data = pd.read_sql_query("SELECT * FROM Non_US_data", connection)

# Get total counts
us_data_count = len(us_data)
non_us_data_count = len(non_us_data)

# Print total counts
print(f"Total number of records in US_data: {us_data_count}")
print(f"Total number of records in Non_US_data: {non_us_data_count}")

# Get state counts for US_data
state_counts = us_data['state'].value_counts().reset_index()
state_counts.columns = ['State', 'Count']

# Print state counts
print("Total number of records grouped by state in US_data:")
print(state_counts.to_string(index=False))


connection.close()

Total number of records in US_data: 5545
Total number of records in Non_US_data: 196
Total number of records grouped by state in US_data:
               State  Count
          California    660
          Washington    341
            Colorado    308
            New York    296
            Michigan    268
               Texas    243
        Pennsylvania    236
             Florida    221
              Oregon    215
      North Carolina    212
                Ohio    201
            Virginia    174
            Illinois    162
           Wisconsin    140
           Minnesota    132
       Massachusetts    119
             Indiana    111
            Missouri    102
             Arizona     91
               Maine     87
            Maryland     81
          New Jersey     79
           Tennessee     79
         Connecticut     67
             Montana     64
                Iowa     61
             Georgia     58
          New Mexico     57
      South Carolina     57
       New Hampshire  